In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import seaborn as sns
import glob
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras import models
from tensorflow.keras.models import Sequential

In [43]:
def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        if root.find('object'):
            for member in root.findall('object'):
                bbx = member.find('bndbox')                
                xmin = round(float(bbx.find('xmin').text))
                ymin = round(float(bbx.find('ymin').text))
                xmax = round(float(bbx.find('xmax').text))
                ymax = round(float(bbx.find('ymax').text))
                label = member.find('name').text
                value = (root.find('filename').text,
                        int(root.find('size')[0].text),
                        int(root.find('size')[1].text),
                        label,
                        xmin,
                        ymin,
                        xmax,
                        ymax
                        )
                xml_list.append(value)

    column_name = ['filename', 'width', 'height',
                   'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xmlDf = pd.DataFrame(xml_list, columns=column_name, index=None)
    return xmlDf

In [47]:
df = xml_to_csv("data/annotations/")

train = df.iloc[:(int)(len(df) * 0.7), :]
train.to_csv("data/train_data.csv", index=None)
test = df.iloc[(int)(len(df) * 0.3):, :]
test.to_csv("data/test_data.csv", index=None)

training_image_label = pd.read_csv("data/train_data.csv")
train_images = []
train_targets = []
train_labels = []

testing_image_label = pd.read_csv("data/train_data.csv")
test_images = []
test_targets = []
test_labels = []

In [48]:
for index, row in training_image_label.iterrows():
    (filename, width, height, class_name, xmin, ymin, xmax, ymax) = row
    
    train_image_fullpath = os.path.join("data/images/", filename)
    train_img = keras.preprocessing.image.load_img(train_image_fullpath, target_size=(height, width))
    train_img_arr = keras.preprocessing.image.img_to_array(train_img)
    
    # Bounding Box Coordinate
    xmin = round(xmin/ width, 2)
    ymin = round(ymin/ height, 2)
    xmax = round(xmax/ width, 2)
    ymax = round(ymax/ height, 2)
    
    train_images.append(train_img_arr)
    train_targets.append((xmin, ymin, xmax, ymax))
    train_labels.append(class_name)
    
for index, row in testing_image_label.iterrows():
    (filename, width, height, class_name, xmin, ymin, xmax, ymax) = row
    
    test_image_fullpath = os.path.join("data/images/", filename)
    test_img = keras.preprocessing.image.load_img(test_image_fullpath, target_size=(height, width))
    test_img_arr = keras.preprocessing.image.img_to_array(test_img)
    
    # Bounding Box Coordinate
    xmin = round(xmin/ width, 2)
    ymin = round(ymin/ height, 2)
    xmax = round(xmax/ width, 2)
    ymax = round(ymax/ height, 2)
    
    test_images.append(train_img_arr)
    test_targets.append((xmin, ymin, xmax, ymax))
    test_labels.append(class_name)
    

In [3]:
#create the common input layer
input_shape = (400, 400, 3)
input_layer = tf.keras.layers.Input(input_shape)

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10)  # Output layer with 10 units (number of classes)
])

# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Train the model
model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels))

# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)

/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


NameError: name 'train_images' is not defined